In [5]:
from pycaret.classification import *
import pandas as pd
from ydata_profiling import ProfileReport 

#dataset 
from pycaret.datasets import get_data
df = pd.read_csv('/home/alpaco/sryang/DACON/submission.csv')

In [7]:
df.shape, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15004 entries, 0 to 15003
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      15004 non-null  object
 1   target  15004 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 234.6+ KB


((15004, 2), None)

In [6]:
profile = df.profile_report()
profile 

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
data = df.sample(frac = 0.95, random_state=42) 
data_test = df.drop(data.index) 

data.reset_index(inplace=True, drop=True)
data_test.reset_index(inplace=True, drop=True)

print("data for model : {}".format(data.shape)) 
print("data for valid : {}".format(data_test.shape)) 

data for model : (14254, 2)
data for valid : (750, 2)


In [13]:
import pandas as pd

train_df = pd.read_csv('/home/alpaco/sryang/DACON/train.csv') 
test_df  = pd.read_csv('/home/alpaco/sryang/DACON/test.csv')

In [ ]:

def assert_dataset_ok(train_df, test_df, df):
    id_col, target_col = df.columns.tolist()
    assert target_col in train_df.columns, f"train_df에 '{target_col}' 컬럼이 없습니다."
    assert id_col in test_df.columns, f"test_df에 '{id_col}' 컬럼이 없습니다."
    assert target_col not in test_df.columns, "
    assert train_df.shape[1] > 2, "train_df에 feature가 없습니다."
    assert test_df.shape[1] > 2, "test_df에 feature가 없습니다."
    print("[OK] 데이터 구조 정상:", train_df.shape, test_df.shape, list(df.columns))

assert_dataset_ok(train_df, test_df, df)

from pycaret.classification import setup, compare_models, finalize_model, predict_model, create_model

id_col, target_col = df.columns.tolist()

s = setup(
    data=train_df,
    target=target_col,
    session_id=21693,
    ignore_features=[id_col], 
    fold=5,
    fold_shuffle=True,
    verbose=False   
)

best = compare_models(sort="F1")
final_model = finalize_model(best)

# 3) 예측
pred = predict_model(final_model, data=test_df)

label_cols = ["Label", "prediction_label", "Prediction"]
score_cols = ["Score", "prediction_score", "Probability"]
pred_col = next((c for c in label_cols if c in pred.columns), None)
if pred_col is None:
    pred_col = next((c for c in score_cols if c in pred.columns), None)
    if pred_col is None:
        raise RuntimeError(f"예측 컬럼을 찾지 못했습니다: {pred.columns}")

[OK] 데이터 구조 정상: (21693, 54) (15004, 53) ['ID', 'target']


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:19:50
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.8689,0.9902,0.8689,0.8658,0.8664,0.8623,0.8625,0.0640
xgboost,Extreme Gradient Boosting,0.7852,0.9759,0.7852,0.7972,0.7900,0.7745,0.7747,3.0360
gbc,Gradient Boosting Classifier,0.7776,0.9744,0.7776,0.7868,0.7795,0.7665,0.7670,92.7780
rf,Random Forest Classifier,0.7607,0.9694,0.7607,0.7715,0.7630,0.7487,0.7492,2.1580
et,Extra Trees Classifier,0.7495,0.9645,0.7495,0.7561,0.7510,0.7370,0.7372,0.7000
dt,Decision Tree Classifier,0.6983,0.8416,0.6983,0.7035,0.7004,0.6832,0.6832,0.3700
nb,Naive Bayes,0.6573,0.9554,0.6573,0.6706,0.6590,0.6402,0.6408,0.0580
knn,K Neighbors Classifier,0.5284,0.8445,0.5284,0.5524,0.5331,0.5048,0.5066,0.1880
lr,Logistic Regression,0.5172,0.8665,0.5172,0.4859,0.4961,0.4931,0.4943,2.2980
lda,Linear Discriminant Analysis,0.4824,0.8628,0.4824,0.5087,0.4881,0.4566,0.4580,0.0980


KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path


TRAIN_PATH = '/home/alpaco/sryang/DACON/train.csv'
TEST_PATH  = '/home/alpaco/sryang/DACON/test.csv'
SAMPLE_SUB_PATH = '/home/alpaco/sryang/DACON/submission.csv'   
SUBMISSION_OUT = 'submission_qda.csv'

USE_PYCARET = True

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)

assert len(sample_sub.columns) == 2, f"sample_submission.csv 컬럼 수=2 가정이 깨졌습니다: {list(sample_sub.columns)}"
ID_COL, TARGET_COL = sample_sub.columns.tolist()


if USE_PYCARET:
    try:
        from pycaret.classification import setup, create_model, finalize_model, predict_model

        ignore_feats = [c for c in [ID_COL] if c in train_df.columns]

        s = setup(
            data=train_df,
            target=TARGET_COL,
            session_id=21693,    
            ignore_features=ignore_feats,
            fold=5,
            fold_shuffle=True,
            verbose=False      
        )

        qda_model = create_model('qda')
        final_model = finalize_model(qda_model)  

        # 예측
        preds = predict_model(final_model, data=test_df)
        label_candidates = ["Label", "prediction_label", "Prediction"]
        pred_col = next((c for c in label_candidates if c in preds.columns), None)
        if pred_col is None:
            score_candidates = ["Score", "prediction_score", "Probability"]
            pred_col = next((c for c in score_candidates if c in preds.columns), None)
            if pred_col is None:
                raise RuntimeError(f"예측 컬럼을 찾지 못했습니다: {list(preds.columns)}")
        if ID_COL in preds.columns:
            out = sample_sub[[ID_COL, TARGET_COL]].merge(
                preds[[ID_COL, pred_col]], on=ID_COL, how='left'
            )
            out[TARGET_COL] = out[pred_col]
            out = out[[ID_COL, TARGET_COL]]
        else:
            if len(sample_sub) != len(preds):
                raise ValueError("예측 결과에 ID가 없고 행 수도 샘플과 다릅니다.")
            out = sample_sub.copy()
            out[TARGET_COL] = preds[pred_col].values

        out.to_csv(SUBMISSION_OUT, index=False)
        print(f"[완료][PyCaret] 저장: {Path(SUBMISSION_OUT).resolve()}")
    except Exception as e:
        print(f"[WARN][PyCaret 경로 실패] {e}")
        USE_PYCARET = False

if not USE_PYCARET:
    from sklearn.pipeline import Pipeline
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    from sklearn.impute import SimpleImputer
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import f1_score

    X = train_df.drop(columns=[TARGET_COL])
    y = train_df[TARGET_COL].copy()

    if ID_COL in X.columns:
        X = X.drop(columns=[ID_COL])
    X_test = test_df.drop(columns=[ID_COL]) if ID_COL in test_df.columns else test_df.copy()

    num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
    cat_cols = [c for c in X.columns if not pd.api.types.is_numeric_dtype(X[c])]

    numeric_tf = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    categorical_tf = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_tf, num_cols),
            ("cat", categorical_tf, cat_cols)
        ],
        remainder="drop"
    )

    # QDA 모델
    qda = QuadraticDiscriminantAnalysis(store_covariance=False)

    pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", qda)
    ])

    try:
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=21693)
        f1s = []
        for tr_idx, va_idx in skf.split(X, y):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
            pipe.fit(X_tr, y_tr)
            y_pred = pipe.predict(X_va)
            avg = 'binary' if y.nunique() == 2 else 'macro'
            f1s.append(f1_score(y_va, y_pred, average=avg))
        print(f"[CV] F1 평균: {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
    except Exception as e:
        print(f"[경고] CV 중 오류(진행은 계속): {e}")

    pipe.fit(X, y)

    yhat_test = pipe.predict(X_test)

    out = sample_sub.copy()
    out[TARGET_COL] = yhat_test
    out = out[[ID_COL, TARGET_COL]]
    out.to_csv(SUBMISSION_OUT, index=False)
    print(f"[완료][scikit-learn] 저장: {Path(SUBMISSION_OUT).resolve()}")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8640,0.9899,0.8640,0.8597,0.8606,0.8572,0.8574
1,0.8703,0.9908,0.8703,0.8652,0.8669,0.8638,0.8639
2,0.8683,0.9898,0.8683,0.8668,0.8668,0.8617,0.8618
3,0.8670,0.9908,0.8670,0.8636,0.8642,0.8603,0.8605
4,0.8749,0.9897,0.8749,0.8739,0.8736,0.8686,0.8687
Mean,0.8689,0.9902,0.8689,0.8658,0.8664,0.8623,0.8625
Std,0.0036,0.0005,0.0036,0.0047,0.0043,0.0038,0.0038


[완료][PyCaret] 저장: /home/alpaco/sryang/DACON/submission_qda.csv


In [18]:
pd.read_csv('submission_qda.csv').head()

,ID,target
0,TEST_00000,4
1,TEST_00001,5
2,TEST_00002,0
3,TEST_00003,9
4,TEST_00004,9
